This notebook guides through the process of semi-manually segmenting the background of the Transmission channel. The obtained mask sparsifies the image data significantly and is needed for computing astar distances through the microstructure. First load the Transmission channel into napari. Perform prewitt edge detection, gaussian denoising, binarize using otsu, run binary closing. Then comes the manual part where one needs to close all the edges. Once the has been done, use flood to segement the PDMS structure channels. Two masks are saved, one with the explant wel included, one with them excluded.

In [1]:
import napari
from tifffile import imread
import numpy as np

from skimage import filters
from skimage import morphology
from skimage.filters import threshold_otsu
from skimage.segmentation import flood


In [38]:
%gui qt

inp_path = '/run/media/loaloa/lbb_ssd/timelapse13_processed/'
# outp_path = '/run/media/loaloa/lbb_ssd/timelapse13_processed/'
tranl_file = inp_path+'D07_G019_Transmission_compr.deflate.tif'
transl_ch = imread(tranl_file)[0]

In [39]:
with napari.gui_qt():
    viewer = napari.Viewer()

    viewer.add_image(transl_ch)
    viewer.layers['transl_ch'].blending='additive'
    viewer.layers['transl_ch'].contrast_limits = [.007*2**16, .0191*2**16]
    viewer.layers['transl_ch'].opacity = .7
    viewer.layers['transl_ch'].gamma = .72

In [40]:
# edge detection
prewitt = filters.prewitt(transl_ch)
viewer.add_image(prewitt)

<Image layer 'prewitt' at 0x7f17159a26a0>

In [41]:
# perform guassian smoothing on edge filtered image
prewitt_gaussian = filters.gaussian(prewitt, sigma=1)
viewer.add_image(prewitt_gaussian)

<Image layer 'prewitt_gaussian' at 0x7f17154ef8b0>

In [42]:
# binarize image
prewitt_bin = prewitt > threshold_otsu(prewitt_gaussian)
viewer.add_image(prewitt_bin)

<Image layer 'prewitt_bin' at 0x7f17154efaf0>

In [43]:
# Run this, then use the bucket tool to fill everything. 
# Play with sigma and solem diameter if things look bad.
# delete parts ofthe image where channels almost touch each other
diameter = 4
selem = morphology.selem.square(diameter) 
prewitt_bin_closed = morphology.binary_closing(prewitt_bin, selem)
viewer.add_labels(prewitt_bin_closed, name='inital_mask')

<Labels layer 'inital_mask' at 0x7f171547c3a0>

In [32]:
# Flood (1000,2700) is the start point
prewitt_bin_closed = viewer.layers['inital_mask'].data
prewitt_bin_closed_flooded = flood(prewitt_bin_closed, (50  ,2000))
# fill wells afterwards and drawy cutoff segmentation
viewer.add_labels(prewitt_bin_closed_flooded, name='before_clean')
# viewer.add_labels(np.zeros_like(prewitt), name='y_cutoff')

<Labels layer 'before_clean' at 0x7f1717bb6070>

In [33]:
prewitt_bin_closed_flooded = viewer.layers['before_clean'].data
selem = morphology.selem.diamond(5)
final = filters.median(prewitt_bin_closed_flooded, selem)
# final fixing
viewer.add_labels(final, name='final')

<Labels layer 'final' at 0x7f1715ebb820>

In [34]:
np.save(tranl_file.replace('.tif', '_mask1.npy'), viewer.layers['before_clean'].data.astype(bool))
